In [1]:
import pandas as pd

import numpy as np
import time

import re
import json
import urllib.request

# def get_rate():
#     url = "http://webforex.hermes.hexun.com/forex/quotelist?code=FOREXUSDCNY&column=Code,Price"
#     req = urllib.request.Request(url)
#     f = urllib.request.urlopen(req)
#     html = f.read().decode("utf-8")
   

#     s = re.findall("{.*}",str(html))[0]
#     sjson = json.loads(s)

#     USDCNY = sjson["Data"][0][0][1]/10000
#     return (USDCNY)

def get_rate():
   

    t = time.time()
    # timestamp must be ms
    timestamp = (int(round(t*1000)))
    url = "http://www.chinamoney.com.cn/r/cms/www/chinamoney/data/fx/ccpr.json?t="+str(timestamp)
    req = urllib.request.Request(url)
    f = urllib.request.urlopen(req)
    html = f.read().decode("utf-8")
    rjson = json.loads(html)
    

    for r in rjson['records']:
        if r['vrtEName'] == 'USD/CNY':
            return float(r['price'])

def convert_currency(value):
    """
    转换字符串数字为float类型
     - 移除 ￥ ,
     - 转化为float类型
    """
    new_value = value.replace(',', '').replace('**',"").replace('*',"").replace('$', '')
   # print(new_value)
    return  np.float(new_value)

def get_data(sym):
    url="https://coinmarketcap.com/zh/currencies/"+sym+"/#markets"
    data =pd.read_html(url)[0]
   
    data['supply']=data['交易量（24小时）'].apply(convert_currency)

    data['price']=data['价格'].apply(convert_currency)

    data['exchange']=data['来源']
    data['sym']=sym

    df = pd.DataFrame(data)
    x=[0,1,2,3,4,5,6,7,8]
    df.drop(df.columns[x], axis=1, inplace=True)
 
   

    #df.drop(['B', 'C'], axis=1)
    return df

 

df=get_data("0x")
 
df=df.loc[df['exchange'].isin(['Poloniex','Huobi','HitBTC','CoinTiger','Binance','Gate.io','OKEx','Ethfinex','Upbit','Bittrex','Bithumb','Coinbase Pro','BitMart','DigiFinex'])]
df



dict_mapping = {
                'price':'mean',
                'supply':'sum'}
dd=df.groupby(['exchange','sym']).agg(dict_mapping)

r=get_rate()
print('当前汇率:',r)
def rate(val): 
    return r*val

dd['cny_supply']=dd['supply'].apply(rate).apply(lambda x: '%.2f' % x)

dd['cny_price']=dd['price'].apply(rate).apply(lambda x: '%.2f' % x)

dd.sort_values(by=['supply'])

dd.T
dd.to_csv('0x_result.csv')
dd

当前汇率: 6.9306


,,price,supply,cny_supply,cny_price
exchange,sym,,,,
Binance,0x,0.350128,2290309.0,15873215.56,2.43
BitMart,0x,0.309645,4023837.0,27887604.71,2.15
Bithumb,0x,0.365128,189962.0,1316550.64,2.53
Bittrex,0x,0.355642,135172.0,936823.06,2.46
CoinTiger,0x,0.353434,208639.0,1445993.45,2.45
Coinbase Pro,0x,0.347909,2816385.0,19519237.88,2.41
DigiFinex,0x,0.352201,576308.0,3994160.22,2.44
Ethfinex,0x,0.352311,874818.0,6063013.63,2.44
Gate.io,0x,0.354865,142650.0,988650.09,2.46
